In [ ]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report

import numpy as np
import pandas as pd
import os
import io
import time

## prepare the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
DATA_DIR = '/content/drive/My Drive/DLDATA/Crop/'

Mounted at /content/drive


In [ ]:
class AttentionModel(tf.keras.Model):
    def __init__(self, input_dim, hidden_dim, output_dim, recurrent_layers, dropout_p):
        super(AttentionModel, self).__init__()

        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.input_dim = input_dim
        self.recurrent_layers = recurrent_layers
        self.dropout_p = dropout_p

        self.input_embeded = tf.keras.layers.Dense(
            hidden_dim//2, activation='tanh')

        self.dropout = tf.keras.layers.Dropout(dropout_p)

        self.rnn_layers = []
        for _ in range(0, recurrent_layers):

            rnn_layer = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(hidden_dim,
                                                                           return_sequences=True,
                                                                           return_state=True))
            self.rnn_layers.append(rnn_layer)

        self.self_attention = tf.keras.Sequential([
            tf.keras.layers.Dense(hidden_dim*2, activation='relu'),
            tf.keras.layers.Dense(1)
        ])

        self.scale = 1.0/np.sqrt(hidden_dim)

        self.output_linear = tf.keras.layers.Dense(self.hidden_dim)
        self.label = tf.keras.layers.Dense(output_dim)

    def call(self, input_sentences):

        input = self.input_embeded(input_sentences)
        input = self.dropout(input)

        for i, _ in enumerate(self.rnn_layers):
            output, forward_h, forward_c, backward_h, backward_c = self.rnn_layers[i](
                input)

        attn_ene = self.self_attention(output)

        # scale
        attn_ene = tf.math.scalar_mul(self.scale, attn_ene)
        attns = tf.nn.softmax(attn_ene, axis=1)

        context_vector = attns * output
        final_inputs = tf.reduce_sum(context_vector, axis=1)
        final_inputs2 = tf.reduce_sum(output, axis=1)

        combined_inputs = tf.concat([final_inputs, final_inputs2], axis=1)

        logits = self.label(combined_inputs)

        return logits

In [ ]:
# model hyperparameters
INPUT_DIM = 1
OUTPUT_DIM = 7
HID_DIM = 128
DROPOUT = 0.2
RECURRENT_Layers = 1
EPOCHS = 400
BATCH_SIZE = 128
num_classes = 7
lr = 0.004
num_gpu = 2
SEED=15

# generate dataloader
data_path_x_train = f'{DATA_DIR}/train_x_small.csv'
data_path_y_train = f'{DATA_DIR}/train_y_small.csv'
data_path_x_test = f'{DATA_DIR}/test_x_small.csv'
data_path_y_test = f'{DATA_DIR}/test_y_small.csv'

# data_path_x_train = f'{DATA_DIR}/train_x_large.csv'
# data_path_y_train = f'{DATA_DIR}/train_y_large.csv'
# data_path_x_test = f'{DATA_DIR}/test_x_large.csv'
# data_path_y_test = f'{DATA_DIR}/test_y_large.csv'


df_x_train = pd.read_csv(data_path_x_train)
df_y_train = pd.read_csv(data_path_y_train, dtype=np.int32)
df_x_test = pd.read_csv(data_path_x_test)
df_y_test = pd.read_csv(data_path_y_test, dtype=np.int32)



df_x_train = pd.concat([df_x_train, df_x_test], axis=0)
df_y_train = pd.concat([df_y_train, df_y_test], axis=0)


train_X, test_X, train_y, test_y = df_x_train.to_numpy(
), df_x_test.to_numpy(), df_y_train.to_numpy(), df_y_test.to_numpy()

train_y = np.squeeze(train_y)
test_y = np.squeeze(test_y)


X_train, X_test, y_train, y_test = train_test_split(
    train_X, train_y, test_size=0.1, random_state=SEED, stratify=train_y)

X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)

print('X_train:{}'.format(X_train.shape))
print('X_train:{}'.format(X_train))
print('X_test:{}'.format(X_test.shape))
print('y_train:{}'.format(y_train))
print('y_train:{}'.format(y_train.shape))
print('y_test:{}'.format(y_test.shape))


model = AttentionModel(INPUT_DIM, HID_DIM,
                           OUTPUT_DIM, RECURRENT_Layers, DROPOUT)


early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                patience=10,
                                                mode='min',restore_best_weights=True)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=f'{DATA_DIR}/small_model.h5',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            optimizer=tf.optimizers.Adam(),
            metrics=['accuracy'])




baseline_history = model.fit(
    X_train,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks = [early_stopping,model_checkpoint_callback],
    validation_data=(X_test, y_test))



X_train:(21781, 41, 1)
X_train:[[[ 0.04916661]
  [ 0.06928869]
  [ 0.08682726]
  ...
  [ 0.125618  ]
  [ 0.11803508]
  [ 0.1488349 ]]

 [[ 0.07300766]
  [ 0.07839973]
  [ 0.0820749 ]
  ...
  [ 0.2463104 ]
  [ 0.17560796]
  [ 0.1086869 ]]

 [[ 0.05702595]
  [ 0.08256717]
  [ 0.09899703]
  ...
  [ 0.09075005]
  [ 0.08141035]
  [-0.06551325]]

 ...

 [[ 0.02729875]
  [ 0.05372446]
  [ 0.07897268]
  ...
  [ 0.12870689]
  [ 0.12182612]
  [ 0.19791627]]

 [[ 0.06541488]
  [ 0.07677525]
  [ 0.08008936]
  ...
  [ 0.17040706]
  [ 0.1244928 ]
  [ 0.08673806]]

 [[ 0.2587482 ]
  [ 0.2848584 ]
  [ 0.30300355]
  ...
  [ 0.24640791]
  [ 0.24452345]
  [ 0.22923668]]]
X_test:(2421, 41, 1)
y_train:[1 1 4 ... 1 1 6]
y_train:(21781,)
y_test:(2421,)
Epoch 1/400
171/171 [==============================] - 3s 18ms/step - loss: 1.0634 - accuracy: 0.5498 - val_loss: 0.8145 - val_accuracy: 0.6766
Epoch 2/400
171/171 [==============================] - 2s 10ms/step - loss: 0.6574 - accuracy: 0.7047 - val_loss: 0.

In [ ]:
# Filtered data
data_path = '/content/drive/My Drive/DLDATA/Crop/cleaned_data_25753.csv'
df_all = pd.read_csv(data_path)

# pick up only NDVI,and paddocktyp
labels = df_all.iloc[:,2]
df_all = df_all.iloc[:, 12:].copy()
mask = (df_all <= 1).all(axis=1)
df_all = df_all[mask]
labels = labels[mask]
X = df_all
y = labels


le = LabelEncoder()
le.fit(y)
print(le.classes_)
class_names = le.classes_
y = le.transform(y)


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=15, stratify=y)


X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)

print('X_train:{}'.format(X_train.shape))
print('X_train:{}'.format(X_train))
print('X_test:{}'.format(X_test.shape))
print('y_train:{}'.format(y_train))
print('y_train:{}'.format(y_train.shape))
print('y_test:{}'.format(y_test.shape))

# balance data
# ros = RandomOverSampler(random_state=SEED)
# X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

X_train_resampled, y_train_resampled = X_train, y_train

# check sample no.
# unique_elements, counts_elements = np.unique(y_train, return_counts=True)
# print("Frequency of unique values of the said array:")
# print(np.asarray((unique_elements, counts_elements)))



# DataLoader definition
# model hyperparameters
INPUT_DIM = 1
OUTPUT_DIM = 7
HID_DIM = 128
DROPOUT = 0.2
RECURRENT_Layers = 1
# LR = 0.001  # learning rate
EPOCHS = 400
BATCH_SIZE = 256
num_classes = 5



# unique_elements, counts_elements = np.unique(y_train, return_counts=True)
# weights = [1/i for i in counts_elements]
# weights[2] = weights[2]/15
# print(np.asarray((unique_elements, counts_elements)))
# print(weights)
# samples_weight = np.array([weights[t] for t in y_train])
# samples_weights = torch.FloatTensor(samples_weight).to(device)
# class_weights = torch.FloatTensor(weights).to(device)
# sampler = torch.utils.data.sampler.WeightedRandomSampler(samples_weights, len(X_train_resampled),replacement=True)

# prepare PyTorch Datasets


model = AttentionModel(INPUT_DIM, HID_DIM,
                           OUTPUT_DIM, RECURRENT_Layers, DROPOUT)


early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                patience=10,
                                                mode='min')


model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            optimizer=tf.optimizers.Adam(),
            metrics=['accuracy'])


baseline_history = model.fit(
    X_train,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks = [early_stopping],
    validation_data=(X_test, y_test))


['Barley' 'Canola' 'Chick Pea' 'Lentils' 'Wheat']
X_train:(22851, 276, 1)
X_train:[[[0.19980183]
  [0.19323996]
  [0.1869162 ]
  ...
  [0.1343711 ]
  [0.13878852]
  [0.14370161]]

 [[0.17000577]
  [0.1663594 ]
  [0.16286805]
  ...
  [0.15502408]
  [0.16259387]
  [0.17065004]]

 [[0.3475954 ]
  [0.34541732]
  [0.3431764 ]
  ...
  [0.12736091]
  [0.1284678 ]
  [0.12981576]]

 ...

 [[0.1295554 ]
  [0.1318962 ]
  [0.1340546 ]
  ...
  [0.14080852]
  [0.1423608 ]
  [0.14403331]]

 [[0.22148573]
  [0.21736622]
  [0.21320131]
  ...
  [0.15086061]
  [0.15091068]
  [0.15094978]]

 [[0.1399711 ]
  [0.14132032]
  [0.14262459]
  ...
  [0.15678623]
  [0.15832445]
  [0.16002485]]]
X_test:(2540, 276, 1)
y_train:[4 4 4 ... 0 2 0]
y_train:(22851,)
y_test:(2540,)
Epoch 1/400
715/715 [==============================] - 17s 24ms/step - loss: 1.4393 - accuracy: 0.3832 - val_loss: 1.3231 - val_accuracy: 0.4547
Epoch 2/400
 78/715 [==>...........................] - ETA: 13s - loss: 1.2539 - accuracy: 0.4119

KeyboardInterrupt: ignored

In [ ]:
# checkpoint_dir = './training_checkpoints'
# checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
# checkpoint = tf.train.Checkpoint(optimizer=optimizer,
#                                  encoder=encoder,
#                                  decoder=decoder)